# Introdução a Machine Learning nas Geociências
**Ministrante: Marcos Jacinto, Geólogo, Mestrando e Geocientista de Dados na Geowellex**

[LinkedIn](https://www.linkedin.com/in/marcos-jacinto/)

O presente curso fornece uma introdução aos conceitos de Machine Learning, utilizando a linguagem python e a plataforma Google Colaboratory como ambiente de programação. 

O intuito do uso do Google Colaboratory é evitar a obrigação de instalar um software no computador de cada participante. Com isso podemos evitar eventuais problemas de instalação e configuração, e, ao mesmo tempo, acelerar o estudo dos conceitos aqui apresentados.

O objetivo do nosso curso é:

*   Adquirir conhecimentos básicos sobre o workflow necessário para executar um trabalho de Machine Learning, aplicando-o para predição de litologias em poços;
*   Aprender o funcionamento básico de alguns métodos de Machine Learning;
*   Aprender a avaliar e criticar os resultados obtidos.

O roteiro da parte prática será:

1.   Como os Dados são vistos pelo computador?
2.   Preparação dos Dados;
3.   Treinando Diferentes Modelos

---
Antes de iniciar os primeiros passos, tenha certeza de ter colocado<b> esse notebook junto com demais arquivos que foram enviados na raíz do seu Google Drive, dentro de uma pasta com o nome</b>: 

*   Curso AAPG

É importante que o nome seja exatamente esse para evitar problemas de carregamento dos dados. Se a pasta não estiver localizada na raíz ou o nome estiver ligeiramente diferente o carregamento dos dados não irá funcionar como esperado.


---

A primeira coisa que iremos fazer é <b> configurar o Google Colaboratory para utilizar uma GPU</b> na nuvem. Isso deve ser feito para que quando utilizarmos uma rede neural artificial o treinamento seja mais rápido. Repita os seguintes passos:


*   No canto superior esquerdo clique em Editar;
*   Escolha a opção 'Configurações do Notebook';
*   Em 'Acelerador de Hardware' selecione GPU e depois clique em Salvar.

---

Uma vez feito isso, podemos proceder para a nossa parte prática. 
Os códigos estarão dívidos em células. Ao se executar uma célula, somente o código dentro dela será executado. Não se preocupem com as linhas de código abaixo e seus significados, pois o que vocês deverão saber será explicado ao longo do texto. Como exemplo, <b>o que iremos fazer abaixo é importar para o nosso ambiente a maior parte das funcionalidades que iremos precisar durante o minicurso </b>.

Para executar clique no ícone do lado esquerdo referente a 'Executar célula' ou clique dentro dele (onde há texto/código) e aperte Control + Enter.




In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
import time

In [ ]:
import tensorflow as tf
# tf.compat.v1.disable_v2_behavior() # REMOVE FOR COURSE
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report
from matplotlib.colors import ListedColormap
from sklearn.metrics import confusion_matrix

seed = np.random.seed(42)

## Como os Dados são vistos pelo computador?

Conhecer os dados e suas representações é essencial para quem deseja trabalhar com Machine Learning. Entender detalhes sobre os mesmos nos permite identificar se os dados podem ou não serem utilizados junto com algum modelo de Machine Learning, ou, caso necessário, entender o processamento que seria necessário para adequar os dados ao formato necessário.

Dentro do contexto de Machine Learning, iremos trabalhar com dados estruturados ou não-estruturados. 


> Os dados estruturados são organizados e representados como uma estrutura rígida, pensada previamente. Um exemplo são os próprios arquivos LAS, padrão da indústria para registrar dados de poços, que contém uma estrutura rígida onde cada curva é previamente planejada. 

> Os dados não estruturados não possuem uma estrutura bem definida, podendo ser flexíveis ou dinâmicos em sua estrutra. Um exemplo de dado não estruturado é um documento de texto. Não sabemos previamente quantas linhas, caracteres e páginas um texto terá, por exemplo.

Trabalhar com dados estruturados é geralmente mais fácil, devido a sua estrutura bem definida e conhecida. Vejamos um pouco mais sobre dados abaixo.


### Matrizes

Uma matriz é um exemplo de dado estruturado e pode ser entendida como uma tabela que contém um certo número de linhas (m) e um certo número de colunas (n), em que cada termo da matriz pode ser representado pela notação $ A_{i,j}$, onde i representa o número da linha e j o número da coluna. Abaixo temos uma representação genérica de uma matriz.

\begin{bmatrix}
A_{1,1} & A_{1,2} & ... & A_{1,n} \\
A_{2,1} & A_{2,2} & ... & A_{1,n} \\
... & ... & ... &... \\
A_{m,1} & A_{m,2} & ... & A_{m,n}
\end{bmatrix}

No nosso contexto, iremos usualmente trabalhar com dados que podem ser representados como uma matriz, são as tabelas usuais, a exemplo da representação de dados de curvas de geofísica de poço obtidos a partir de um arquivo LAS, ou ainda dados de geoquímica, dentre outras possibilidades.

Vejamos também abaixo uma matriz gerada aleatoriamente.

In [ ]:
matriz = np.random.randint(0, 100, size = (20, 3))
print(matriz)

Vamos representar essa matriz como uma tabela, em um formato que estamos habituados, em que cada coluna da matriz representa uma variável, por exemplo.

In [ ]:
pd.DataFrame(matriz, columns = ['Variável 1', 'Variável 2', 'Variável 3'])

### Imagens

Imagens podem ser consideradas como dados não estruturados, e seu entendimento é importante no ramo das geociências, pois podemos eventualmente trabalhar com seções sísmicas ou imagens de satélite, por exemplo. 

As imagens são formadas por matrizes, onde cada elemento da matriz representa o valor do pixel da imagem.  Se a imagem estiver em preto e branco, uma única matriz é suficiente para descrever os tons de cinza da imagem. Caso a imagem esteja em RGB, serão necessários três matrizes, em que a primeira descreva os valores no canal Red, a segunda, os valores no canal Green, e a terceira, no canal Blue.

Vejamos abaixo um exemplo de uma imagem aleatória, em que foram geradas três matrizes para descrevê-la, configurando uma imagem com os canais RGB, por exemplo.

In [ ]:
imagem = np.random.randint(0, 255, size = (12, 12, 3))
imagem[..., 0]

Vejamos agora cada canal representado separadamente, e, por fim, a imagem completa, resultado dos três canais.

In [ ]:
fig, ax = plt.subplots(1, 4, figsize = (30, 10), sharey = True)

ax[0].imshow(imagem[..., 0], cmap = 'gray') # Primeiro canal da imagem
ax[0].set_title('Primeiro Canal')
ax[1].imshow(imagem[..., 1], cmap = 'gray') # Segundo canal da imagem
ax[1].set_title('Segundo Canal')
ax[2].imshow(imagem[..., 2], cmap = 'gray') # Terceiro canal da imagem
ax[2].set_title('Terceiro Canal')
ax[3].imshow(imagem) # Imagem completa com todos os canais
ax[3].set_title('Imagem Completa');

for (j,i),label in np.ndenumerate(imagem[..., 0]):
    ax[0].text(i, j, label, ha='center', va='center')

for (j,i),label in np.ndenumerate(imagem[..., 1]):
    ax[1].text(i, j, label, ha='center', va='center')

for (j,i),label in np.ndenumerate(imagem[..., 2]):
    ax[2].text(i, j, label, ha='center', va='center')
    
fig.tight_layout()

Caso trabalhemos com uma seção sísmica, por exemplo, cada pixel na imagem vai representar o valor da amplitude da onda. Se gerarmos atributos, cada pixel irá representar aquele dado transformado. Se gerarmos cinco atributos, por exemplo, serão necessárias 6 matrizes para descrever a seção e os demais atributos.

## Preparação dos Dados

**A qualidade do seu modelo preditivo irá depender não só do tipo de abordagem (algoritmo) que você escolheu, mas principalmente da qualidade do seus dados.** Assim, uma fase extremamente importante é a preparação dos dados, que pode ser subdividida em diversas subetapas, a exemplo: exploração/visualização dos dados, limpeza de dados, criação de novas features/variáveis, normalização dos dados, e estratégias de separação de teste e treino. Essas etapas serão discutidas adiantes, mas antes vamos fazer uma breve reflexão sobre a fase geral de preparação de dados.


Um exemplo, fora das geociências, sobre a importância da fase de preparação de dados está em AIs que discriminam pessoas com base em sua renda ou ainda em sua cor de pele. [Um caso](https://www.technologyreview.com/2019/01/21/137783/algorithms-criminal-justice-ai/), por exemplo, pode ser citado através dos controversos algoritmos de avaliação de risco criminal aplicados nos EUA.

> "As ferramentas de avaliação de risco são projetadas para fazer uma coisa: tomar os detalhes do perfil de um réu e cuspir uma pontuação de reincidência - um único número estimando a probabilidade de que ele ou ela reincidirá. Um juiz então leva em conta essa pontuação em uma quantidade inumerável de decisões que podem determinar que tipo de serviços de reabilitação determinados réus devem receber, se eles devem ser mantidos na prisão antes do julgamento e quão severas suas sentenças devem ser. Uma pontuação baixa abre caminho para um destino mais amável. Uma pontuação alta faz exatamente o oposto."

> [...] algoritmos de aprendizado de máquina usam estatísticas para encontrar padrões nos dados. Portanto, se você alimentá-lo com dados históricos de crimes, ele selecionará os padrões associados ao crime. Mas esses padrões são correlações estatísticas - nem de longe o mesmo que causações. Se um algoritmo descobrir, por exemplo, que a baixa renda está correlacionada com a alta reincidência, não ficaria sabendo se a baixa renda realmente causou o crime. Mas é exatamente isso que as ferramentas de avaliação de risco fazem: elas transformam percepções correlativas em mecanismos de pontuação causal.

E nas geociências que tipo de erros podem ser gerados por uma fase de preparação dos dados inadequada?

[Outro link sobre esse tipo de assunto, caso queiram saber mais](https://www.technologyreview.com/2020/07/17/1005396/predictive-policing-algorithms-racist-dismantled-machine-learning-bias-criminal-justice/)

Na célula abaixo, iremos realizar a leitura do nosso conjunto de dados, em seguida passaremos por diversas partes do workflow.

In [ ]:
caminho = 'https://github.com/marcosjacinto/MLIntroGeoscience/blob/master/data.zip?raw=true'
dataset = pd.read_csv(caminho, compression='zip')

A célula abaixo irá nos informar o número de poços diferentes no nosso conjunto de dados.

In [ ]:
numeroDePocos = len(list(np.unique(dataset.WELL)))
print(f'Temos nesse dataset {numeroDePocos} poços diferentes')

### Explorando e Visualizando os Dados

Iniciaremos nosso worfklow observando as algumas características do dataset e das suas variáveis.

O primeiro passo é observar quais são as variáveis presentes que podem nos auxiliar para montar nosso modelo preditivo.

O dataset contém as seguintes informações:
* WELL: nome do poço;
* DEPTH_MD: profundidade medida;
* X_LOC: coordenada UTM X; 
* Y_LOC: coordenada UTM Y;
* Z_LOC: profundidade Z;
* GROUP: grupo litoestratigráfico;
* FORMATION: formação litoestratigráfica;

As colunas a seguir, contidas no dataset, são referentes às curvas de geofísica de poço, dentre outras curvas:
* BS: tamanho da broca;
* CALI: caliper;
* RDEP: resistividade profunda;
* RHOB: densidade;
* GR: raios gama;
* SGR: raios gama espectral;
* RMED: resistividade média;
* ROP: taxa de penetração;
* NPHI: porosidade neutrão;
* PEF: fator de absorção fotoelétrico;
* RSHA: resistividade rasa;
* DTS: sônico (cisalhante)
* DTC: sônico (compressional)

Possui ainda colunas referentes a litologia, o que queremos predizer:
* LITHOLOGY: tipo de rocha;
* CONFIDENCE: confiança na interpretação dessa litologia (1: alta, 2: média, 3: baixa).

A célula abaixo irá nos mostrar um exemplo de como estão as colunas no nosso dataset, e como está seu preenchimento.

In [ ]:
dataset.sample(5)

É importante também ter uma noção básica da distribuição dos valores de cada variável. A célula abaixo nos mostrará alguns parâmetros estatísticos, onde:
* count: contagem do número de registros dessa variável;
* mean: valor médio;
* std: desvio-padrão;
* min: valor mínimo:
* 25%: percentil 25;
* 50%: percentil 50 ou mediana;
* 75%: percentil 75;
* max: valor máximo.

In [ ]:
dataset.describe().T

Algo que devemos fazer é observar o comportamento dessas curvas nos poços. As células abaixo nos permitem visualizar os diferentes poços contidos no nosso dataset.

Primeiramente, execute a célula abaixo para criar uma função que será utilizada pra visualizar os poços.

In [ ]:
from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1 import make_axes_locatable

cmap = ListedColormap(['yellow', 'yellowgreen', 'forestgreen', 'springgreen', 'teal',
                       'steelblue', 'darkslategray', 'paleturquoise', 'darkturquoise',
                       'saddlebrown', 'grey', 'orangered'])

def plotarPoco(dataFrame, numeroDoPoco):

    nomeDoPoco = dataFrame.WELL.unique()[numeroDoPoco]

    well = dataFrame[dataFrame.WELL == nomeDoPoco]

    assert numeroDoPoco < len(dataFrame.WELL.unique()), f'Inserir número do poço entre 0 e {len(dataFrame.WELL.unique())}'

    lithology_keys = {0: ['Sandstone', 'yellow'],
                 1: ['Sandstone/Shale', 'yellowgreen'],
                 2: ['Shale', 'forestgreen'],
                 3: ['Marl', 'springgreen'],
                 4: ['Dolomite', 'teal'],
                 5: ['Limestone', 'steelblue'],
                 6: ['Chalk', 'darkslategray'],
                 7: ['Halite', 'paleturquoise'],
                 8: ['Anhydrite', 'darkturquoise'],
                 9: ['Tuff', 'saddlebrown'],
                 10: ['Coal', 'grey'],
                 11: ['Basement', 'orangered']}

    fig, ax = plt.subplots(1, 6, figsize = (30, 15), sharey = False)

    ax[0].plot(well.CALI, well.DEPTH_MD, color = 'black',
             label = 'Caliper')
    ax[0].plot(well.BS, well.DEPTH_MD, color = 'red',
           label = 'Diâmetro da Broca')
    #ax[0].invert_yaxis()
    ax[0].grid()
    ax[0].set_title('Caliper')

    ax[1].set_title('Gamma Ray')
    ax[1].plot(well.GR, well.DEPTH_MD, color = 'green',
           label = 'Gamma Ray', linewidth = '0.5')
    ax[1].set_xlim(0, 150)
    axtwin1 = ax[1].twiny()
    axtwin1.plot(well.GR, well.DEPTH_MD, 'g--',
           label = 'Gamma Ray', linewidth = 0.5)
    axtwin1.set_xlim(150, 300)
    ax[1].grid()
    #ax[1].invert_yaxis()

    ax[2].set_title('Resistividade')
    ax[2].semilogx(well.RDEP, well.DEPTH_MD, color = 'r',
           label = 'Res. Profunda', linewidth = '0.5')
    ax[2].semilogx(well.RMED, well.DEPTH_MD, color = 'm',
           label = 'Res. Média', linewidth = '0.5')
    ax[2].semilogx(well.RSHA, well.DEPTH_MD, color = 'k',
           label = 'Res. Rasa', linewidth = '0.5')
    ax[2].set_xlim(0.2 , 2000)
    #ax[2].set_xticks([0.2, 2, 20, 200, 2000])
    ax[2].grid()
    #ax[2].invert_yaxis()
    ax[2].legend()

    ax[3].set_title('Sônico')
    ax[3].plot(well.DTC, well.DEPTH_MD, color = 'k',
             label = 'Sônico', linewidth = '0.5')
    ax[3].set_xlim(40,240)
    ax[3].grid()
    #ax[3].invert_yaxis()
    ax[3].invert_xaxis()

    ax[4].set_title('Densidade', color = 'r', fontsize = 12)
    ax[4].plot(well.NPHI, well.DEPTH_MD,
             label = 'Neutrão', color = 'b', linewidth = '0.5')
    ax[4].tick_params(axis='x', labelcolor='b')
    ax[4].set_xlim(-0.15, 0.45)
    ax[4].set_xticks(np.linspace(-0.15,0.45,4))
    ax[4].set_xlabel('Neutrão', color = 'b', fontsize = 12)
    ax[4].invert_xaxis()
    axtwin4 = ax[4].twiny()
    axtwin4.plot(well.RHOB, well.DEPTH_MD,
           label = 'Densidade', color = 'r', linewidth = '0.5')
    axtwin4.tick_params(axis='x', labelcolor= 'r')
    axtwin4.set_xlim(2, 3)
    axtwin4.set_xticks(np.linspace(2,3,5))  
    axtwin4.grid()
    ax[4].grid()
    #ax[4].invert_yaxis()

    im = ax[5].imshow(np.array(well.LITHOLOGY).reshape(-1, 1), aspect = 'auto',
               vmin = 0, vmax = 11, cmap = cmap,)
              #  extent = [0, 1, well.DEPTH_MD.iloc[-1], well.DEPTH_MD.iloc[0]])
    divider = make_axes_locatable(ax[5])
    cax = divider.append_axes("right", size="20%", pad=0.05)
    cbar=plt.colorbar(im, cax=cax, cmap = cmap)

    cbar.set_label((16*' ').join(['SS', 'SS-Sh', 'Sh', 
                               ' Ml', 'Dm', 'LS', 'Chk ', 
                                ' Hl', 'Ann', 'Tuf', 'Coal', 'Bsmt']))
    cbar.set_ticks(range(0,1)); cbar.set_ticklabels('')


    #ax[5].legend()

    ax[0].set_ylim(well.DEPTH_MD.iloc[0], well.DEPTH_MD.iloc[-1])
    ax[1].set_ylim(well.DEPTH_MD.iloc[0], well.DEPTH_MD.iloc[-1])
    ax[2].set_ylim(well.DEPTH_MD.iloc[0], well.DEPTH_MD.iloc[-1])
    ax[3].set_ylim(well.DEPTH_MD.iloc[0], well.DEPTH_MD.iloc[-1])
    ax[4].set_ylim(well.DEPTH_MD.iloc[0], well.DEPTH_MD.iloc[-1])
    ax[5].set_xticks([])
    ax[5].set_yticks([])

    ax[0].invert_yaxis()
    ax[1].invert_yaxis()
    ax[2].invert_yaxis()
    ax[3].invert_yaxis()
    ax[4].invert_yaxis()
    #ax[5].invert_yaxis()

Agora, execute a célula abaixo para escolher um número referente a um dos 96 poços E, em seguida, execute a próxima célula, para visualizar os dados do respectivo poço.

In [ ]:
numeroDoPoco = widgets.IntSlider(min = 1, max = 96, step = 1,
                                 description = 'Número do poço')
print('Selecione abaixo o número do poço:')
display(numeroDoPoco)

In [ ]:
plotarPoco(dataset, numeroDoPoco.value - 1)

Execute a célula abaixo para observar a distribuição dos poços em 2D e 3D.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig,ax = plt.subplots(nrows = 1, ncols = 2, figsize = (14, 7))
ax[1].remove()
ax[1]=fig.add_subplot(1, 2, 2,projection='3d')

ax[0].set_xlabel('X Coord.')
ax[0].set_ylabel('Y Coord.')
ax[0].set_title('Distribuição dos poços em mapa')
ax[0].grid()

ax[1].set_xlabel('X Coord.')
ax[1].set_ylabel('Y Coord.')
ax[1].set_zlabel('Depth')
ax[1].set_title('Distribuição dos poços em 3D')
ax[1].set_xticks(range(420000, 560000, 30000))

for nomeDoPoco in dataset.WELL.unique():
    well = dataset[dataset.WELL == nomeDoPoco]
    ax[1].plot(well.X_LOC, well.Y_LOC, well.Z_LOC, color = 'black')
    ax[0].plot(well.X_LOC, well.Y_LOC, 'kx')

fig.tight_layout()

Devemos observar ainda quais as rochas existentes nos nossos poços e sua ocorrência relativa. A partir disso, é importante discutir quais problemas podem surgir devido a uma ocorrência baixa de um tipo de rocha, havendo também um alto número de ocorrências de outro(s) tipo(s) de rocha.

In [ ]:
lithology_keys = {
    0: 'Sandstone',
    1: 'Sandstone/Shale',
    2: 'Shale',
    3: 'Marl',
    4: 'Dolomite',
    5: 'Limestone',
    6: 'Chalk',
    7: 'Halite',
    8: 'Anhydrite',
    9: 'Tuff',
    10: 'Coal',
    11: 'Basement'
}


counts = dataset['LITHOLOGY'].value_counts()
names = []
percentage = []
N = dataset['LITHOLOGY'].shape[0]
for item in counts.iteritems():
    names.append(lithology_keys[item[0]])
    percentage.append(float(item[1])/N*100)
fig, ax = plt.subplots(1, 1, figsize=(14, 7))
ax.bar(x=np.arange(len(names)), height=percentage, color = 'k')
ax.set_xticklabels(names, rotation=45)
ax.set_xticks(np.arange(len(names)))
ax.set_ylabel('Litologia em porcentagem');

### Criação de novas features/variáveis

É interessante se perguntar se as variáveis que estão no nosso conjunto de dados são suficientes para modelar o nosso problema. Além disso, devemos levar em conta quais modelos iremos utilizar. Modelos de Deep Learning, possuem a capacidade de durante o treinamento "extrair" novas features automaticamente, isso é exemplificado por exemplo em redes neurais convolucionais. O link exemplifica um caso do que uma rede neural convolucional aplicada à previsão de números aprende durante o treinamento.

[2D Visualization of a Convolutional Neural Network](https://www.cs.ryerson.ca/~aharley/vis/conv/flat.html)

Modelos que não pertencem à classe de Deep Learning, por outro lado, não possuem a mesma capacidade, e, por consequência, se beneficiam bastante da chamada fase de <i>feature engineering</i>. Nessa fase é nosso papel gerar novas features/variáveis a partir do que já existe no nosso conjunto de dados atuais. Dentre os tipos de conhecimentos que podem nos ajudar nessa fase está o entendimento do próprio problema. Se estamos trabalhando com dados de poços, tipos de features novas que podem ser criadas podem ser similares ao que seria feito numa processo de interpretação petrofísica, por exemplo calculando valores de porosidade ou volume de folhelho.

Por isso, é importante a participação de um geólogo ou geofísico no processo, pois o mesmo poderá identificar como obter novas features, e terá maior capacidade para criticar esse procedimento bem como os resultados obtidos.

Outras features que podem gerados, por outro lado, podem depender de conhecimentos não necessariamente geológicos, mas matemáticos e/ou estatísticos, por exemplo. Isso justifica, por exemplo, a importância de demais profissionais envolvidos em projetos de Machine Learning. 

No exemplo abaixo, não iremos gerar uma variável resultante de uma interpretação petrofísica. Iremos criar uma <i>lag feature</i>. Não se preocupem muito com esse termo, basta entender que a partir da coluna de litologia iremos criar uma nova coluna que vai guardar qual foi a litologia anterior. Execute a célula abaixo para criar três colunas:


1.   Lag feature guardando a litologia imediatamente anterior;
2.   Lag feature guardando a décima litologia anterior;
3.   Lag feature guardando a centésima litologia anterior.




In [ ]:
pocos = dataset.WELL.unique()

dataset['LAG_LITO_1'] = -99
dataset['LAG_LITO_10'] = -99
dataset['LAG_LITO_100'] = -99

for poco in pocos:
    dataset.loc[dataset.loc[:, 'WELL'] == poco, 'LAG_LITO_1'] = dataset.loc[dataset.loc[:, 'WELL'] == poco, 'LITHOLOGY'].shift(1)
    dataset.loc[dataset.loc[:, 'WELL'] == poco, 'LAG_LITO_10'] = dataset.loc[dataset.loc[:, 'WELL'] == poco, 'LITHOLOGY'].shift(10)
    dataset.loc[dataset.loc[:, 'WELL'] == poco, 'LAG_LITO_100'] = dataset.loc[dataset.loc[:, 'WELL'] == poco, 'LITHOLOGY'].shift(100)

Vejamos agora como fica o nosso banco de dados:

In [ ]:
dataset.sample(15)

### Limpeza dos Dados

A limpeza de dados é uma das etapas mais importantes no workflow de um trabalho de Machine Learning. É importante ter qualidade nos dados, pois não importa o quão sofisticado e avançado seja o modelo que você vai criar, sem dados de qualidade, eles não serão realmente válidos ou mesmo irão retornar um reultado medíocre.

---

O tipo de limpeza de dados que será executada dependerá do problema que está sendo resolvido e tambem dos tipos de dados com que se está trabalhando. Para essa etapa também é fundamental compreender o máximo possível sobre os dados sendo utilizados. 

Alguns exemplos do que pode ser realizado nessa etapa são: 

1.   Tratamento de valores anômalos;
2.   Checagem de validade dos dados;
3.   Modificação do formato dos dados;
4.   Padronização dos dados;
5.   Tratamento de valores ausentes.

Dentre outros.

#### Apresentando os NaNs

<center> 
Lixo entra, lixo sai.
</center> 

<figure>
<center>
<img src='https://imgs.xkcd.com/comics/machine_learning.png' />
<figcaption>Muita atenção com os dados que estão sendo utilizados [1]. </figcaption></center>
</figure>

O que é um NaN? Essa sigla significa **Not a Number**, e é usada para representar um valor número indefinido ou que não pode ser representado. A raiz quadrada de um número negativo ou uma divisão por zero, são exemplos de operações que retornariam *NaNs* como resultados.

A exemplo de dados de poços, os NaNs podem aparecer no intervalo de profundidade que a ferramenta não realizou medidas ou mesmo quando a ferramenta geofísica falha e não registra um valor a uma dada profundidade. No arquivo LAS ocorre um valor nulo, geralmente representado pelo valor -999.25.

No python ele em geral é representado por:

```
np.nan
```

Abaixo iremos exemplificar algumas situações relacionadas aos NaNs e em seguida sua importância dentro do contexto de Machine Learning.

[[1] - Referência da figura](https://subscription.packtpub.com/book/application_development/9781788838535/1/ch01lvl1sec12/garbage-in-garbage-out)

In [ ]:
print('Realizando operações que resultam em NaN:')
print(f'Ao calcular a raíz quadrada de -1, o valor retornado é: {np.sqrt(-1)}')
print(f'Ao calcular 10 dividido por 0, o valor retornado é: {np.divide(10,0)}')

Observem que um dos resultados retornou o valor 'inf', resultado de uma divisão por zero. Esse valor também irá prejudicar os modelos preditivos e deve ser evitado.

In [ ]:
exemplo = np.array([1, 2, 3, 4, 5])
print('Array de exemplo:')
print(exemplo)

In [ ]:
print('Somando cada elemento da array com um NaN:')
print(f'{exemplo} + {np.nan} = {exemplo + np.nan}')

In [ ]:
print('Multiplicando cada elemento da array por um NaN:')
print(f'{exemplo} * {np.nan} = {exemplo * np.nan}')

Como os NaNs não são exatamente números, cálculos que os envolvam geram também NaNs. Observem a partir dos exemplos acima que os NaNs ao interagirem com outros números (somar, multiplicar ou qualquer outra operação) irão ser 'propagados'. 

Para Machine Learning, que depende inteiramente de operações matemáticas, os NaNs são indesejáveis. Algumas possibilidades de como resolver a presença de NaNs na base de dados são:
- Eliminar as observações que contém NaNs;
- Substituir NaNs por algum valor como zero, mediana, média, o valor imediatamente anterior, ou o valor mais comum, por exemplo.

#### Observando os NaNs dentro da base de dados

Primeiro vamos observar a cobertura de dados na base de treinos. Abaixo, veremos o nome da coluna representando uma variável diferente, e onde há a presença de dados veremos um preenchimento cinza. Por outro lado, onde há ausência de dados, ou seja, ocorrem NaNs, veremos branco no lugar do cinza.



In [ ]:
msno.bar(dataset);

In [ ]:
msno.matrix(dataset);

#### Tratando os NaNs

Uma vez que observamos a presença e a distribuição de NaNs na base de dados, temos algumas perguntas a fazer:

1. Quais colunas (variáveis) iremos utilizar passar os nossos modelos realizarem as predições?
2. Das colunas que escolhemos iremos eliminar as observações que contêm ausência de informação (NaN) ou iremos preencher com algum valor?
3. Se escolhermos eliminar as observações com NaNs, como fica a distribuição de classes? Todas as classes estão presentes ainda ou alguma foi eliminada?

Então, baseando-se no que discutimos até agora. Quais colunas vocês usariam?

In [ ]:
datasetLimpo = dataset.drop(
    columns = [
        'SGR', 'DTS', 'DCAL', 'MUDWEIGHT', 'RMIC',
        'ROPA', 'RXO', 'RSHA', 'ROP', 'BS', 'PEF'
    ]
)

Vamos eliminar também colunas que não pretendemos utilizar durante o treinamento.

In [ ]:
datasetLimpo = datasetLimpo.drop(columns = ['GROUP', 'FORMATION'])
datasetLimpo = datasetLimpo.drop(columns = ['X_LOC', 'Y_LOC', 'Z_LOC', 'CONFIDENCE'])

Veja o efeito disso no nosso conjunto de dados:

In [ ]:
datasetLimpo.sample(15)

Vamos tratar os NaNs que surgiram com a criação das lag features referentes à litologia, de um modo diferente através de uma técnica chamada one-hot encoding.

Imagine a seguinte representação das litologias através de números:

\begin{bmatrix}
0 \\ 
1 \\
2 \\
2 \\
1
\end{bmatrix}

Onde 0 representa a primeira litologia, 1 a segunda litologia, e 2 a terceira litologia. Através de one-hot encoding transformamos o vetor acima em uma matriz, cuja coluna agora representa a litologia, o 0 representa ausência e 1 indica presença. Veja que não perdemos nenhuma das informações acima, apenas mudamos como representá-las:

\begin{bmatrix}
1 & 0 & 0 \\ 
0 & 1 & 0 \\
0 & 0 & 1 \\
0 & 0 & 1 \\
0 & 1 & 0
\end{bmatrix}

Imagine agora a presença de um NaN na coluna de litologias:

\begin{bmatrix}
0 \\ 
1 \\
2 \\
NaN \\
1
\end{bmatrix}

Transformando com one-hot encoding eliminamos-o:


\begin{bmatrix}
1 & 0 & 0 \\ 
0 & 1 & 0 \\
0 & 0 & 1 \\
0 & 0 & 0 \\
0 & 1 & 0
\end{bmatrix}

Execute a célula abaixo para realizar essa transformação, e depois veja como ficou o nosso dataset.


In [ ]:
dataset = pd.get_dummies(datasetLimpo, columns = ['LAG_LITO_1', 'LAG_LITO_10', 'LAG_LITO_100'])
dataset.sample(15)

Vejamos agora o nosso dataset, como se encontra:

In [ ]:
msno.matrix(dataset);

Iremos tratar o restante dos NaNs eliminando as linhas que contém algum NaN.

In [ ]:
tamanhoInicial = dataset.shape[0]
finalDataset = dataset.dropna()
tamanhoFinal = finalDataset.shape[0]
num = len(list(np.unique(finalDataset.WELL)))

print(f'Agora temos {num} poços.') 
print(f'O tamanho inicial de exemplos era de {tamanhoInicial} e agora temos {tamanhoFinal} exemplos.')
print(f'Isso representa um redução de {100 - (100*tamanhoFinal/tamanhoInicial):.2f}%')

Iremos eliminar também o nome do poço, pois ele não nos fornece uma informação necessariamente importante para realizar a previsão das litologias.

In [ ]:
finalDataset.drop(columns = ['WELL'], inplace = True)

Feito isso, vamos verificar novamente como está a nossa base de dados e visualizar a ocorrência de NaNs na mesma.

In [ ]:
msno.matrix(finalDataset);

### Separando em Teste e Treino

Vejamos abaixo algumas estratégias que devemos considerar para podermos avaliar a performance dos modelos que iremos desenvolver.



1.   <b>Estratégia 1</b>

<center> 
A estratégia mais básica de separação, separar o dataset original no Treino e Teste.
</center> 

<figure>
<center>
<img src='https://developers.google.com/machine-learning/crash-course/images/PartitionTwoSets.svg' />
<img src='https://developers.google.com/machine-learning/crash-course/images/WorkflowWithTestSet.svg' />
<figcaption> Avaliamos a performance no conjunto de teste, e escolhemos o melhor resultado em função disso [2, 3]. </figcaption></center>
</figure>

2.   <b>Estratégia 2</b>

<center> 
Uma estratégia mais robusta, separar o dataset original no Teste, Validação e Teste.
</center> 

<figure>
<center>
<img src='https://developers.google.com/machine-learning/crash-course/images/PartitionThreeSets.svg' />
<img src='https://developers.google.com/machine-learning/crash-course/images/WorkflowWithValidationSet.svg' />
<figcaption>Avaliamos a performance no conjunto de validação, e escolhemos o melhor resultado em função disso [2, 3]. </figcaption></center>
</figure>

É importante ressaltar que quando se trata de dados geológicos, alguns fatores poderiam ser considerados durante a separação. Por exemplo, poderíamos separar o conjunto de teste considerando todos os dados ou considerando os poços. No primeiro caso, aleatoriamente partes diferentes de poços seriam selecionadas para teste. No segundo caso, selecionaríamos aleatoriamente poços inteiros para testes. Porém, deve se ter certeza que durante a separação, tenhamos todos os tipos de litologias tanto no treino quanto no teste.

[[2] - Referência da Figura](https://developers.google.com/machine-learning/crash-course/training-and-test-sets/splitting-data)

[[3] - Referência da Figura](https://developers.google.com/machine-learning/crash-course/validation/another-partition)

In [ ]:
from sklearn.model_selection import train_test_split

treino, teste = train_test_split(finalDataset, test_size = 0.2, stratify = finalDataset.LITHOLOGY)

# Explicar as diferentes estratégias
target = ['LITHOLOGY']

feature_names = treino.drop(columns = target).columns

xtreino = np.array(treino.drop(columns = target))
xteste = np.array(teste.drop(columns = target))
ytreino = np.array(treino.loc[:, 'LITHOLOGY'])
yteste = np.array(teste.loc[:, 'LITHOLOGY'])

Executando as células acima nós criamos as seguintes separações:


*   X Treino - Variáveis preditoras do conjunto de treino;
*   Y Treino - Variável resposta (litologia) do conjunto de treino;
*   X Teste - Variáveis preditoras do conjunto de teste;
*   Y Teste - Variável resposta (litologia) do conjunto de teste;



### Normalização dos dados

Uma etapa importante é a normalização dos dados. Seu objetivo é adequar os valores dos dados a uma escala comum, porém sem distorcer as diferenças na amplitude dos valores. Isso é particularmente importante no aprendizado de redes neurais, as quais veremos no final desse minicurso.

Abaixo iremos utilizar uma estratégia que vai normalizar os dados entre os valores 0 e 1, através do cálculo:

<center> $ \frac{Valor - Mínimo}{Máximo - Mínimo} $  </center>

Esse cálculo é realizado para cada variável preditora, obtendo cada máximo e mínimo e depois transformando os dados usando a fórmula acima. Execute a célula abaixo para realizar essa etapa.

In [ ]:
scaler = preprocessing.MinMaxScaler()

scaler.fit(np.concatenate([xtreino]))

xtreino = scaler.transform(xtreino)
xteste = scaler.transform(xteste)

## Treinando Diferentes Modelos

Com os dados devidamente preparados, iremos agora para parte da modelagem, onde testaremos e compararemos alguns métodos de machine learning. A princípio executamos a célula abaixo que nos ajudará posteriormente para visualizar os resultados, e em seguida partiremos para criação dos modelos em si.

In [ ]:
def plotarPredicao(numeroDoPoco, modelo, redeNeural = False):

    fig, ax = plt.subplots(1, 3, figsize = (20, 15), sharey = False)

    pocos = dataset.WELL.unique()
    data = dataset.loc[dataset.WELL == pocos[numeroDoPoco], feature_names].copy()
    x = scaler.transform(data)

    mask = np.where((data.isnull().sum(axis = 1)) >= 1, True, False)

    if redeNeural:
        prediction = np.zeros((x.shape[0])) * np.nan
        if prediction[~mask].shape[0] == 0:
            print('Ao eliminar os NaNs, não restou nenhuma amostra do poço')
        else:
            prediction[~mask] = np.argmax(modelo.predict(x[~mask]), axis = 1)

    else:
        prediction = np.zeros((x.shape[0])) * np.nan
        if prediction[~mask].shape[0] == 0:
            print('Ao eliminar os NaNs, não restou nenhuma amostra do poço')
        else: 
            prediction[~mask] = modelo.predict(x[~mask])

    real = np.expand_dims(dataset.loc[dataset.WELL == pocos[numeroDoPoco], 'LITHOLOGY'].copy(), axis = 1)

    ax[0].set_title('Gamma Ray')
    ax[0].plot(data.GR, data.DEPTH_MD, color = 'green',
           label = 'Gamma Ray', linewidth = '0.5')
    ax[0].set_xlim(0, 150)
    axtwin1 = ax[0].twiny()
    axtwin1.plot(data.GR, data.DEPTH_MD, 'g--',
           label = 'Gamma Ray', linewidth = 0.5)
    axtwin1.set_xlim(150, 300)
    ax[0].set_ylim(data.DEPTH_MD.iloc[0], data.DEPTH_MD.iloc[-1])
    ax[0].grid(which = 'both')

    ax[1].imshow(real, aspect = 'auto', cmap = cmap, vmin = 0, vmax = 11,
               extent = [0, 1, data.DEPTH_MD.iloc[-1], data.DEPTH_MD.iloc[0]])
    im = ax[2].imshow(prediction.reshape(-1, 1), aspect = 'auto', cmap = cmap,
                    vmin = 0, vmax = 11,
                    extent = [0, 1, 0, len(real)])
    ax[2].set_yticks([])

    divider = make_axes_locatable(ax[2])
    cax = divider.append_axes("right", size="20%", pad=0.05)
    cbar = plt.colorbar(im, cax=cax, cmap = cmap)

    cbar.set_label((16*' ').join(['SS', 'SS-Sh', 'Sh', 
                                ' Ml', 'Dm', 'LS', 'Chk ', 
                                ' Hl', 'Ann', 'Tuf', 'Coal', 'Bsmt']))
    cbar.set_ticks(range(0,1)); cbar.set_ticklabels('')

    ax[1].set_title('Litologia Real')
    ax[2].set_title('Litologia Prevista pelo Modelo')
    ax[1].set_xticks([])
    ax[2].set_xticks([])

    ax[0].invert_yaxis()
    
  
def plotarTeste(modelo, litologia, redeNeural = False):

    fig, ax = plt.subplots(1, 2, figsize = (15, 15), sharey = True)

    ypred = modelo.predict(xteste)
    if redeNeural:
        ypred = np.argmax(ypred, axis = 1)

    ax[0].imshow(yteste[yteste == litologia].reshape(-1, 1), vmin = 0, vmax = 11, cmap = cmap,
               aspect = 'auto', extent = [0, 1, 0, len(ypred[yteste == litologia])])

    im = ax[1].imshow(ypred[yteste == litologia].reshape(-1, 1), vmin = 0, vmax = 11, cmap = cmap,
               aspect = 'auto', extent = [0, 1, 0, len(ypred[yteste == litologia])])

    divider = make_axes_locatable(ax[1])
    cax = divider.append_axes("right", size="20%", pad=0.05)
    cbar = plt.colorbar(im, cax=cax, cmap = cmap)

    cbar.set_label((16*' ').join(['SS', 'SS-Sh', 'Sh', 
                                ' Ml', 'Dm', 'LS', 'Chk ', 
                                ' Hl', 'Ann', 'Tuf', 'Coal', 'Bsmt']))
    cbar.set_ticks(range(0,1)); cbar.set_ticklabels('')

    #ax[0].set_yticks([])
    ax[1].set_xticks([])
    ax[0].set_xticks([])
    ax[0].set_title(f'{lithology_keys[litologia]} no Conjunto de Teste')
    ax[1].set_title('Litologia Prevista pelo Modelo para o Teste');

### Classificador Fictício

Um classificador fictício (ou [dummy classifier](https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html)) é um tipo de classificador que utiliza regras simples para realizar previsões. As possiveis estratégias que esse classificador pode utilizar são:
- Estratificado: gera previsões baseadas na distribuição de classes do conjunto de dados de treino;
- Mais frequente: sempre prediz a classe que mais ocorre no conjunto de dados de treino;
- Uniforme: gera previsões de forma aleatória.
- Constante: o classificador sempre irá prever a mesma classe, escolhida pelo usuário.

Ou seja, esse tipo de classificador não busca entender a relação entre as diversas variáveis com as classes que está tentando prever. Ele não deve ser utilizado para resolver problemas reais, e somente serve como uma linha base para comparações.

Execute as células abaixo para treinarmos nosso classificador fictício.





In [ ]:
from sklearn.dummy import DummyClassifier

Sugestão: tentar modificar o modo abaixo (na primeira linha) para alguma das estratégias abaixo:



*   'most_frequent'
*   'stratified'
*   'uniform'



In [ ]:
modo = 'most_frequent' #'most_frequent', 'stratified' ou 'uniform'

dummy_model = DummyClassifier(strategy = modo)

dummy_model.fit(xtreino, ytreino)

Após treinar a célula abaixo nos informará da acurácia do nosso modelo.

In [ ]:
score = dummy_model.score(xteste, yteste)
print(f'O modelo treinado teve uma acurácia de {100*score:.2f}%')

As duas próximas células serão utilizadas para comparar as litologias reais nos poços com as previsões realizadas pelo modelo acima. Selecione um número entre 1 e 96 para ver o respectivo resultado para aquele poço.

In [ ]:
numeroDoPocoPred = widgets.IntSlider(min = 1, max = 96, step = 1,
                                 description = 'Número do poço')
print('Selecione abaixo o número do poço a ser previsto:')
display(numeroDoPocoPred)

In [ ]:
plotarPredicao(numeroDoPocoPred.value - 1, dummy_model)

Devemos lembrar que na estratégia de separação, consideramos todo o conjunto de dados. Logo, alguns poços visualizados acima todos podem conter todo o seu trecho utilizado durante o treino. Isso implica não podermos usá-los como comparativo para avaliar visualmente a performance do nosso modelo. 

Assim, execute as duas células abaixo para observar somente os dados de teste e suas respectivas previsões, comparando litologia a litologia. Note ainda que essa visualização não reflete de forma alguma estratigrafia ou a disposição das litologias em um poço específico.

In [ ]:
numeroLito = widgets.IntSlider(min = 1, max = 12, step = 1,
                                 description = 'Litologia:')
print('Escolha um número entre 1 e 12 refere à litologia, para comparação:')
display(numeroLito)


In [ ]:
plotarTeste(dummy_model, numeroLito.value - 1)

Vimos acima uma métrica numérica para avaliar a performance do nosso modelo: a acurácia. A acurácia compara o total de acertos com o número de exemplos na nosso conjunto de teste que está sendo avaliado através da fórmula:

<center> $ \frac {Total \: de \: acertos} {Total  \:  de  \:  exemplos}$ </center>

O problema dessa métrica é que ela avalia todas as classes/litologias ao mesmo tempo, mas não nos informa individualmente a eficiência de previsão de uma certa litologia. Por isso, é importante também utilizar as métricas Precisão, Recall e F1-Score, definidas do seguinte modo:

<center>
Precisão:

$ \frac {Positivos \: verdadeiros} {Positivos \: verdadeiros \: + \:Falsos \: verdadeiros}$

Recall:

$ \frac {Positivos \: verdadeiros} {Positivos \: verdadeiros \: + \:Falsos \: negativos}$

F1-Score:

$ 2 \cdot \frac {Precisao \:\cdot\: Recall} {Precisao \:+\: Recall}$

</center>

Quanto mais próximos de 1, melhor a eficiência de predição da litologia, e quanto mais próximo de 0, pior. Na dúvida, basta olhar para o F1-Score visto que o mesmo leva em consideração tanto precisão quanto recall para unificar as métricas.

In [ ]:
print(classification_report(yteste, dummy_model.predict(xteste),
                            target_names = lithology_keys.values()))

Uma outra forma de observar os resultados individualmente é a matriz de confusão. Execute a célula abaixo e vamos discutir o significado da matriz de confusão e como realizar sua leitura.

In [ ]:
confusionMatrix = confusion_matrix(yteste, dummy_model.predict(xteste))

fig, ax = plt.subplots(1, 1, figsize = (20 , 12))

sns.heatmap(confusionMatrix, cmap = 'GnBu', annot = True, square = True,
            ax = ax, fmt = 'g')
ax.set_xticklabels(list(lithology_keys.values()), rotation = 'vertical')
ax.set_yticklabels(list(lithology_keys.values()), rotation = 'horizontal')
ax.set_title('Matriz de confusão', fontsize = 14)
ax.set_xlabel('Previsto')
ax.set_ylabel('Verdadeiro');

### Decision Trees (Árvores de Decisão)

[Árvores de decisão](https://scikit-learn.org/stable/modules/tree.html) é um método supervisionado que pode ser utilizado tanto para regressão quanto para classificação. Seu objetivo é criar um modelo que realiza predições a partir de regras simples.

As árvores de decisão requerem pouca preparação de dados. Para esse modelo, algumas fases, como normalização dos dados, não seriam necessárias, por exemplo. Também possui a vantagem de poder ser visualizada, e, consequentemente, interpretável.

Deve-se tomar cuidado para que as árvores geradas não se tornem complexas demais, de modo que o modelo criado não consiga generalizar bem para novos dados.

[Esse vídeo](https://www.youtube.com/watch?v=7VeUPuFGJHk&ab_channel=StatQuestwithJoshStarmer) apresenta um excelente resumo de como funciona uma árvore de decisão e como a mesma é criada. Vamos dar uma olhada e observar alguns pontos, como exemplos de árvores de decisão, a sua estrutura dentre outros.

Execute as duas células abaixo para realizar o treinamento. O tempo esperado de treino é de cerca de 30 segundos.

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [ ]:
# Modifque aqui, com valores acima de 1 ou None:
max_depth = 10

tree_model = DecisionTreeClassifier(max_depth = max_depth, class_weight = 'balanced')

start = time.time()
tree_model.fit(xtreino, ytreino)
end = time.time()
print(f'Treino concluído. Tempo de treino {end - start} s')

Após treinar a célula abaixo nos informará da acurácia do nosso modelo.

In [ ]:
score = tree_model.score(xteste, yteste)
print(f'O modelo treinado teve uma acurácia de {100*score:.2f}%')

As duas próximas células serão utilizadas para comparar as litologias reais nos poços com as previsões realizadas pelo modelo acima. Selecione um número entre 1 e 96 para ver o respectivo resultado para aquele poço.

In [ ]:
numeroDoPocoPred = widgets.IntSlider(min = 1, max = 96, step = 1,
                                 description = 'Número do poço')
print('Selecione abaixo o número do poço a ser previsto:')
display(numeroDoPocoPred)

In [ ]:
plotarPredicao(numeroDoPocoPred.value - 1, tree_model)

Devemos lembrar que na estratégia de separação, consideramos todo o conjunto de dados. Logo, alguns poços visualizados acima todos podem conter todo o seu trecho utilizado durante o treino. Isso implica não podermos usá-los como comparativo para avaliar visualmente a performance do nosso modelo. 

Assim, execute as duas células abaixo para observar somente os dados de teste e suas respectivas previsões, comparando litologia a litologia. Note ainda que essa visualização não reflete de forma alguma estratigrafia ou a disposição das litologias em um poço específico.

In [ ]:
numeroLito = widgets.IntSlider(min = 1, max = 12, step = 1,
                                 description = 'Litologia:')
print('Escolha um número entre 1 e 12 refere à litologia, para comparação:')
display(numeroLito)

In [ ]:
plotarTeste(tree_model, numeroLito.value - 1)

Vimos acima uma métrica numérica para avaliar a performance do nosso modelo: a acurácia. A acurácia compara o total de acertos com o número de exemplos na nosso conjunto de teste que está sendo avaliado através da fórmula:

<center> $ \frac {Total \: de \: acertos} {Total  \:  de  \:  exemplos}$ </center>

O problema dessa métrica é que ela avalia todas as classes/litologias ao mesmo tempo, mas não nos informa individualmente a eficiência de previsão de uma certa litologia. Por isso, é importante também utilizar as métricas Precisão, Recall e F1-Score, definidas do seguinte modo:

<center>
Precisão:

$ \frac {Positivos \: verdadeiros} {Positivos \: verdadeiros \: + \:Falsos \: verdadeiros}$

Recall:

$ \frac {Positivos \: verdadeiros} {Positivos \: verdadeiros \: + \:Falsos \: negativos}$

F1-Score:

$ 2 \cdot \frac {Precisao \:\cdot\: Recall} {Precisao \:+\: Recall}$

</center>

Quanto mais próximos de 1, melhor a eficiência de predição da litologia, e quanto mais próximo de 0, pior. Na dúvida, basta olhar para o F1-Score visto que o mesmo leva em consideração tanto precisão quanto recall para unificar as métricas.

In [ ]:
print(classification_report(yteste, tree_model.predict(xteste),
                            target_names = lithology_keys.values()))

Uma outra forma de observar os resultados individualmente é a matriz de confusão. Execute a célula abaixo e vamos discutir o significado da matriz de confusão e como realizar sua leitura.

In [ ]:
confusionMatrix = confusion_matrix(yteste, tree_model.predict(xteste))

fig, ax = plt.subplots(1, 1, figsize = (20 , 12))

sns.heatmap(confusionMatrix, cmap = 'GnBu', annot = True, square = True,
            ax = ax, fmt = 'g')
ax.set_xticklabels(list(lithology_keys.values()), rotation = 'vertical')
ax.set_yticklabels(list(lithology_keys.values()), rotation = 'horizontal')
ax.set_title('Matriz de confusão', fontsize = 14)
ax.set_xlabel('Previsto')
ax.set_ylabel('Verdadeiro');

Esse modelo nos permite obter uma medida da importância de cada variável para realizar a previsão da litologia. Execute a célula abaixo para gerar um gráfico que nos irá mostrar essas informações.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (20, 10))

feature_importances = tree_model.feature_importances_

ax.bar(range(1, len(feature_names) + 1), height = feature_importances, color = 'k');
ax.grid(axis = 'y')

ax.set_title('Importância de cada variável');
ax.set_ylabel('Valor atribuído de importância pela Árvore de Decisão')
ax.set_xticks(range(1, len(feature_names) + 1))
ax.set_xticklabels(feature_names, rotation = 'vertical')
ax.set_xlabel('Variáveis');

Em particular, o modelo Árvores de Decisão pode ser interpretável. O código abaixo nos mostra a árvore de decisão criada para realizar as previsões. Caso a profundidade da árvore seja muito grande, dificilmente iremos conseguir ver nitidamente a imagem gerada. Para entender esse conceito, sugiro modificar o tamanho da árvore para um numéro pequeno (menor que 5, por exemplo) e só depois executar a célula abaixo. Evite executar a célula caso o tamanho da árvore seja grande.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (30, 30))

class_names = list(lithology_keys.values())

plot_tree(tree_model,
          filled = True,
          class_names = class_names,
          feature_names = feature_names,
          ax = ax);

### Random Forest (Florestas Aleatórias)

As [Florestas Aleatórias ou Random Forests](https://scikit-learn.org/stable/modules/ensemble.html#forests-of-randomized-trees) pertencem ao grupo de métodos <i>ensemble</i>. Esses métodos possuem a característica de combinar vários preditores como base para realizar sua predição, com o objetivo de aumentar a capacidade de generalização e a robustez das previsões. Nesse caso, em específico uma Random Forest é formada por diversas árvores de decisão. Vamos entender um pouco mais sobre esse método abaixo.

Em uma floresta aleatória, cada árvore precisa ser diferente da outra, do contrário todas as árvores fariam exatamente a mesma predição e não faria sentido combinar várias árvores, pois bastaria somente uma. Por isso, existe uma 'aleatoriedade' na construção de cada árvore. Essa aleatoridade ocorre em dois momentos principais:

*   O conjunto de dados inteiro não é utilizado, sendo selecionado aleatoriamente apenas parte do mesmo para a construção de árvore;
*   No momento de escolher a raíz da árvore, e cada nó interno, as opções são limitadas através de uma escolha aleatória dentre as possíveis variáveis preditoras existentes no dataset.

Após construir todas as árvores, no momento da predição no final cada árvore irá ter o seu voto. Desse modo, a classe (litologia) que obtiver mais votos será a escolhida.

[Vamos observar esse video](https://www.youtube.com/watch?v=7VeUPuFGJHk&ab_channel=StatQuestwithJoshStarmer) para entender como são construídas as Florestas Aleatórias.



In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Escolher profundidade e número de estimadores

rf = RandomForestClassifier(n_estimators = 100,
                            criterion = 'gini',
                            max_depth = 3,
                            class_weight = 'balanced',
                            )

In [ ]:
start = time.time()
rf.fit(xtreino, ytreino)
end = time.time()
print(f'Treino concluído. Tempo de treino {(end - start):.2f} s')

Após treinar a célula abaixo nos informará da acurácia do nosso modelo.

In [ ]:
score = rf.score(xteste, yteste)
print(f'O modelo treinado teve uma acurácia de {100*score:.2f}%')

As duas próximas células serão utilizadas para comparar as litologias reais nos poços com as previsões realizadas pelo modelo acima. Selecione um número entre 1 e 96 para ver o respectivo resultado para aquele poço.

In [ ]:
numeroDoPocoPred = widgets.IntSlider(min = 1, max = 96, step = 1,
                                 description = 'Número do poço')
print('Selecione abaixo o número do poço a ser previsto:')
display(numeroDoPocoPred)

In [ ]:
plotarPredicao(numeroDoPocoPred.value - 1, rf)

Devemos lembrar que na estratégia de separação, consideramos todo o conjunto de dados. Logo, alguns poços visualizados acima todos podem conter todo o seu trecho utilizado durante o treino. Isso implica não podermos usá-los como comparativo para avaliar visualmente a performance do nosso modelo. 

Assim, execute as duas células abaixo para observar somente os dados de teste e suas respectivas previsões, comparando litologia a litologia. Note ainda que essa visualização não reflete de forma alguma estratigrafia ou a disposição das litologias em um poço específico.

In [ ]:
numeroLito = widgets.IntSlider(min = 1, max = 12, step = 1,
                                 description = 'Litologia:')
print('Escolha um número entre 1 e 12 refere à litologia, para comparação:')
display(numeroLito)

In [ ]:
plotarTeste(rf, numeroLito.value - 1)

Vimos acima uma métrica numérica para avaliar a performance do nosso modelo: a acurácia. A acurácia compara o total de acertos com o número de exemplos na nosso conjunto de teste que está sendo avaliado através da fórmula:

<center> $ \frac {Total \: de \: acertos} {Total  \:  de  \:  exemplos}$ </center>

O problema dessa métrica é que ela avalia todas as classes/litologias ao mesmo tempo, mas não nos informa individualmente a eficiência de previsão de uma certa litologia. Por isso, é importante também utilizar as métricas Precisão, Recall e F1-Score, definidas do seguinte modo:

<center>
Precisão:

$ \frac {Positivos \: verdadeiros} {Positivos \: verdadeiros \: + \:Falsos \: verdadeiros}$

Recall:

$ \frac {Positivos \: verdadeiros} {Positivos \: verdadeiros \: + \:Falsos \: negativos}$

F1-Score:

$ 2 \cdot \frac {Precisao \:\cdot\: Recall} {Precisao \:+\: Recall}$

</center>

Quanto mais próximos de 1, melhor a eficiência de predição da litologia, e quanto mais próximo de 0, pior. Na dúvida, basta olhar para o F1-Score visto que o mesmo leva em consideração tanto precisão quanto recall para unificar as métricas.

In [ ]:
print(classification_report(yteste, rf.predict(xteste),
                            target_names = lithology_keys.values()))

Uma outra forma de observar os resultados individualmente é a matriz de confusão. Execute a célula abaixo e vamos discutir o significado da matriz de confusão e como realizar sua leitura.

In [ ]:
confusionMatrix = confusion_matrix(yteste, rf.predict(xteste))

fig, ax = plt.subplots(1, 1, figsize = (20 , 12))

sns.heatmap(confusionMatrix, cmap = 'GnBu', annot = True, square = True,
            ax = ax, fmt = 'g')
ax.set_xticklabels(list(lithology_keys.values()), rotation = 'vertical')
ax.set_yticklabels(list(lithology_keys.values()), rotation = 'horizontal')
ax.set_title('Matriz de confusão', fontsize = 14)
ax.set_xlabel('Previsto')
ax.set_ylabel('Verdadeiro');

Esse modelo nos permite obter uma medida da importância de cada variável para realizar a previsão da litologia. Execute a célula abaixo para gerar um gráfico que nos irá mostrar essas informações.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (20, 10))

feature_importances = rf.feature_importances_

ax.bar(range(1, len(feature_names) + 1), height = feature_importances, color = 'k');
ax.grid(axis = 'y')

ax.set_title('Importância de cada variável');
ax.set_ylabel('Valor atribuído de importância pela Árvore de Decisão')
ax.set_xticks(range(1, len(feature_names) + 1))
ax.set_xticklabels(feature_names, rotation = 'vertical')
ax.set_xlabel('Variáveis');

### AdaBoost

[AdaBoost](https://scikit-learn.org/stable/modules/ensemble.html#adaboost) também pertence aos métdos de <i>ensemble</i>, porém seu funcionamento diferente de Random Forests. A ideia principal por trás desse método é ajustar uma sequência de preditores fracos em versões repetidas dos dados modificados. As predições feitas por cada preditor são, então, combinadas através de um peso, diferentemente de Random Forests onde todas as árvores possuem o mesmo peso.

Inicialmente, o peso é igual para todas as árvores, porém eles são modificados sucessivamente, de modo que exemplos incorretamente previstos passam a ter um peso maior. Isso força com que cada preditor fraco subsequente passe a focar em exemplos que foram incorretamente classificados pelos preditores anteriores.

[Vamos observar esse video](https://www.youtube.com/watch?v=LsK-xG1cLYA) para entender como é construído o modelo AdaBoost.

Em seguida, execute as células abaixo para realizar o treinamento do modelo.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
ab = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3,
                                                                class_weight = 'balanced'),
                        n_estimators = 150,)

In [ ]:
start = time.time()
ab.fit(xtreino, ytreino)
end = time.time()
print(f'Tempo de treino {end - start} s')

Após treinar a célula abaixo nos informará da acurácia do nosso modelo.

In [ ]:
score = ab.score(xteste, yteste)
print(f'O modelo treinado teve uma acurácia de {100*score:.2f}%')

As duas próximas células serão utilizadas para comparar as litologias reais nos poços com as previsões realizadas pelo modelo acima. Selecione um número entre 1 e 96 para ver o respectivo resultado para aquele poço.

In [ ]:
numeroDoPocoPred = widgets.IntSlider(min = 1, max = 96, step = 1,
                                 description = 'Número do poço')
print('Selecione abaixo o número do poço a ser previsto:')
display(numeroDoPocoPred)

In [ ]:
plotarPredicao(numeroDoPocoPred.value - 1, ab)

Devemos lembrar que na estratégia de separação, consideramos todo o conjunto de dados. Logo, alguns poços visualizados acima todos podem conter todo o seu trecho utilizado durante o treino. Isso implica não podermos usá-los como comparativo para avaliar visualmente a performance do nosso modelo. 

Assim, execute as duas células abaixo para observar somente os dados de teste e suas respectivas previsões, comparando litologia a litologia. Note ainda que essa visualização não reflete de forma alguma estratigrafia ou a disposição das litologias em um poço específico.

In [ ]:
numeroLito = widgets.IntSlider(min = 1, max = 12, step = 1,
                                 description = 'Litologia:')
print('Escolha um número entre 1 e 12 refere à litologia, para comparação:')
display(numeroLito)

In [ ]:
plotarTeste(ab, numeroLito.value - 1)

Vimos acima uma métrica numérica para avaliar a performance do nosso modelo: a acurácia. A acurácia compara o total de acertos com o número de exemplos na nosso conjunto de teste que está sendo avaliado através da fórmula:

<center> $ \frac {Total \: de \: acertos} {Total  \:  de  \:  exemplos}$ </center>

O problema dessa métrica é que ela avalia todas as classes/litologias ao mesmo tempo, mas não nos informa individualmente a eficiência de previsão de uma certa litologia. Por isso, é importante também utilizar as métricas Precisão, Recall e F1-Score, definidas do seguinte modo:

<center>
Precisão:

$ \frac {Positivos \: verdadeiros} {Positivos \: verdadeiros \: + \:Falsos \: verdadeiros}$

Recall:

$ \frac {Positivos \: verdadeiros} {Positivos \: verdadeiros \: + \:Falsos \: negativos}$

F1-Score:

$ 2 \cdot \frac {Precisao \:\cdot\: Recall} {Precisao \:+\: Recall}$

</center>

Quanto mais próximos de 1, melhor a eficiência de predição da litologia, e quanto mais próximo de 0, pior. Na dúvida, basta olhar para o F1-Score visto que o mesmo leva em consideração tanto precisão quanto recall para unificar as métricas.

In [ ]:
print(classification_report(yteste, ab.predict(xteste),
                            target_names = lithology_keys.values()))

Uma outra forma de observar os resultados individualmente é a matriz de confusão. Execute a célula abaixo e vamos discutir o significado da matriz de confusão e como realizar sua leitura.

In [ ]:
confusionMatrix = confusion_matrix(yteste, ab.predict(xteste))

fig, ax = plt.subplots(1, 1, figsize = (20 , 12))

sns.heatmap(confusionMatrix, cmap = 'GnBu', annot = True, square = True,
            ax = ax, fmt = 'g')
ax.set_xticklabels(list(lithology_keys.values()), rotation = 'vertical')
ax.set_yticklabels(list(lithology_keys.values()), rotation = 'horizontal')
ax.set_title('Matriz de confusão', fontsize = 14)
ax.set_xlabel('Previsto')
ax.set_ylabel('Verdadeiro');

Esse modelo nos permite obter uma medida da importância de cada variável para realizar a previsão da litologia. Execute a célula abaixo para gerar um gráfico que nos irá mostrar essas informações.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (20, 10))

feature_importances = ab.feature_importances_

ax.bar(range(1, len(feature_names) + 1), height = feature_importances, color = 'k');
ax.grid(axis = 'y')

ax.set_title('Importância de cada variável');
ax.set_ylabel('Valor atribuído de importância pela Árvore de Decisão')
ax.set_xticks(range(1, len(feature_names) + 1))
ax.set_xticklabels(feature_names, rotation = 'vertical')
ax.set_xlabel('Variáveis');

### Logistic Regression (Regressão Logística)

Apesar do seu nome, a regressão logística serve para resolver problemas de classificação. Nesse modelo a saída será um valor variando entre 0 a 1, que indica a probabilidade de determinadas features/variáveis preditoras se referirem a uma certa classe, quando comparada com as demais.


<center> 
A saída esperada será similar à figura abaixo:
</center> 
<figure>
<center>
<img src='https://developers.google.com/machine-learning/crash-course/images/LogisticRegressionOutput.svg' />
<figcaption> O valor de saída ficará entre 0 e 1, indicando a probabilidade de pertencer àquela classe [4]. </figcaption></center>
</figure>

[Observemos esse vídeo](https://www.youtube.com/watch?v=yIYKR4sgzI8&ab_channel=StatQuestwithJoshStarmer) para entender um pouco mais sobre a ideia por trás desse método.

Em seguida, execute as células abaixo para treinar o modelo de Regressão Logística.

[[4] - Referência da figura](https://developers.google.com/machine-learning/crash-course/logistic-regression/calculating-a-probability)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
max_iter = 100

lr = LogisticRegression(max_iter = max_iter)
start = time.time()
lr.fit(xtreino, ytreino)
end = time.time()
print(f'Treino concluído. Tempo de treino {end - start} s')
 # Configurar max_iter # Pontuar que quanto mais iterações, mais tempo necessário para treinar, ex 10.000 demora muito

Após treinar a célula abaixo nos informará da acurácia do nosso modelo.

In [ ]:
score = lr.score(xteste, yteste)
print(f'O modelo treinado teve uma acurácia de {100*score:.2f}%')

As duas próximas células serão utilizadas para comparar as litologias reais nos poços com as previsões realizadas pelo modelo acima. Selecione um número entre 1 e 96 para ver o respectivo resultado para aquele poço.

In [ ]:
numeroDoPocoPred = widgets.IntSlider(min = 1, max = 96, step = 1,
                                 description = 'Número do poço')
print('Selecione abaixo o número do poço a ser previsto:')
display(numeroDoPocoPred)

In [ ]:
plotarPredicao(numeroDoPocoPred.value - 1, lr)

Devemos lembrar que na estratégia de separação, consideramos todo o conjunto de dados. Logo, alguns poços visualizados acima todos podem conter todo o seu trecho utilizado durante o treino. Isso implica não podermos usá-los como comparativo para avaliar visualmente a performance do nosso modelo. 

Assim, execute as duas células abaixo para observar somente os dados de teste e suas respectivas previsões, comparando litologia a litologia. Note ainda que essa visualização não reflete de forma alguma estratigrafia ou a disposição das litologias em um poço específico.

In [ ]:
numeroLito = widgets.IntSlider(min = 1, max = 12, step = 1,
                                 description = 'Litologia:')
print('Escolha um número entre 1 e 12 refere à litologia, para comparação:')
display(numeroLito)

In [ ]:
plotarTeste(lr, numeroLito.value - 1)

Vimos acima uma métrica numérica para avaliar a performance do nosso modelo: a acurácia. A acurácia compara o total de acertos com o número de exemplos na nosso conjunto de teste que está sendo avaliado através da fórmula:

<center> $ \frac {Total \: de \: acertos} {Total  \:  de  \:  exemplos}$ </center>

O problema dessa métrica é que ela avalia todas as classes/litologias ao mesmo tempo, mas não nos informa individualmente a eficiência de previsão de uma certa litologia. Por isso, é importante também utilizar as métricas Precisão, Recall e F1-Score, definidas do seguinte modo:

<center>
Precisão:

$ \frac {Positivos \: verdadeiros} {Positivos \: verdadeiros \: + \:Falsos \: verdadeiros}$

Recall:

$ \frac {Positivos \: verdadeiros} {Positivos \: verdadeiros \: + \:Falsos \: negativos}$

F1-Score:

$ 2 \cdot \frac {Precisao \:\cdot\: Recall} {Precisao \:+\: Recall}$

</center>

Quanto mais próximos de 1, melhor a eficiência de predição da litologia, e quanto mais próximo de 0, pior. Na dúvida, basta olhar para o F1-Score visto que o mesmo leva em consideração tanto precisão quanto recall para unificar as métricas.

In [ ]:
print(classification_report(yteste, lr.predict(xteste),
                            target_names = lithology_keys.values()))

Uma outra forma de observar os resultados individualmente é a matriz de confusão. Execute a célula abaixo e vamos discutir o significado da matriz de confusão e como realizar sua leitura.

In [ ]:
confusionMatrix = confusion_matrix(yteste, lr.predict(xteste))

fig, ax = plt.subplots(1, 1, figsize = (20 , 12))

sns.heatmap(confusionMatrix, cmap = 'GnBu', annot = True, square = True,
            ax = ax, fmt = 'g')
ax.set_xticklabels(list(lithology_keys.values()), rotation = 'vertical')
ax.set_yticklabels(list(lithology_keys.values()), rotation = 'horizontal')
ax.set_title('Matriz de confusão', fontsize = 14)
ax.set_xlabel('Previsto')
ax.set_ylabel('Verdadeiro');

### Redes Neurais: MLP

As redes neurais são algoritmos de Machine Learning inspirados no funcionamento dos neurônios e a interconectividade entre os mesmos no cérebro humano. A sua unidade fundamental é o perceptron, como visto na figura abaixo:

<center> 
O perceptron, unidade fundamental em uma rede neural:
<figure>
<img src='https://miro.medium.com/max/510/1*7pwA1DjBw6JDkwZQecUNiw.png' />
<figcaption> Para obter o valor de saída (y), cada valor de entrada Xi é multiplicado por um peso Wi [5]. </figcaption>
</center> 
</figure>

A ideia por trás disso não é nova, mas remonta à decada de 1940. Com o avanço computacional foi possível criar estruturas mais complexas, surgindo então o Multi Layer Perceptron (MLP). Veja abaixo sua estrutura e seus componentes.

<center> 
A estrutura de uma rede neural pode ser esquematizada como abaixo:
</center> 
<figure>
<center>
<img src='https://developers.google.com/machine-learning/crash-course/images/1hidden.svg' />
<figcaption> Rede neural com um hidden layer, com três neurônios cada [6]. </figcaption>
<img src='https://developers.google.com/machine-learning/crash-course/images/2hidden.svg' />
<figcaption> Rede neural com dois hidden layers, com três neurônios cada. Estamos transicionando para Deep Learning nesse caso. [6]. </figcaption></center>
</figure>

As redes neurais possuem duas fases principais:
* Forward Propagation: durante essa fase os valores de entrada (nossas variáveis preditoras) serão multiplicadas pelos seus respectivos pesos, e em seguida passada por uma função de ativação em cada neurônio. Isso se repete para as demais camadas e neurônios da rede, até chegarmos no valor de saída (output);
* [Back Propagation](https://www.youtube.com/watch?v=Ilg3gGewQ5U&ab_channel=3Blue1Brown): durante essa fase a rede neural irá calcular uma função chamada função de custo que vai nos informar o quanto a rede neural errou em suas previsões. A partir desse valor, a rede neural irá reajustar os pesos utilizados, buscando minimizar esse erro, e consequentemente, a função de custo. Pode-se entender que a parte ativa do aprendizado nesse algoritmo ocorre nessa fase.

[Vamos observar através desse link](https://playground.tensorflow.org/) alguns conceitos relativos a redes neurais antes de realizarmos o nosso treinamento.

[[5] - Referência da figura](https://towardsdatascience.com/the-perceptron-3af34c84838c)

[[6] - Referência da figura](https://developers.google.com/machine-learning/crash-course/introduction-to-neural-networks/anatomy)

Execute a célula abaixo para criar uma função que irá nos ajudar a construir o nosso modelo de rede neural.

In [ ]:
def construirModeloNN(neuronios = 16,
                      numeroDeCamadas = 0,
                      neuronios2 = 32,                      
                      saida = 1):
  
  modelo = Sequential()

  modelo.add(Dense(neuronios,
                   activation = 'relu',
                   input_shape = (xtreino.shape[-1], )
                   ))
  #modelo.add(Dropout(rate = 0.2))
  
  for i in range(numeroDeCamadas):
    modelo.add(Dense(neuronios2,
                   activation = 'relu',
                   ))
  
  modelo.add(Dense(saida,
                   activation = 'softmax'))
  
  modelo.compile(loss = 'categorical_crossentropy',
                 optimizer = 'Adam',
                 metrics = ['categorical_accuracy'])

  return modelo

Execute a célula abaixo para escolher o número de camadas ocultas adicionais, o número de neurônios na primeira camada, e o número de neurônios nas camadas adicionais.

In [ ]:
modeloWidget = interactive(construirModeloNN,
                           neuronios = widgets.IntSlider(min = 4, max = 256,
                                                         step = 8,
                                                         description = 'Neurônios-1'),
                           numeroDeCamadas = widgets.IntSlider(min = 0, max = 5,
                                                          step = 1,
                                                          description = 'Camadas Adicionais'),
                           neuronios2 = widgets.IntSlider(min = 8, max = 256,
                                                          step = 8,
                                                          description = 'Neurônios-2'),
                           saida = fixed(12))

display(modeloWidget)

Após escolher as configurações desejadas execute abaixo para criar nosso modelo, e observe as características do mesmo.

In [ ]:
modelo = modeloWidget.result
modelo.summary()

Defina abaixo o número de epochs de treinamento, e quantas amostras serão vistas pela rede neural por vez.

In [ ]:
epochsWidget = widgets.IntSlider(min = 10, max = 100, step = 10,
                                 description = 'Num. Epochs')
batchWidget = widgets.IntSlider(min = 32, max = 1024, step = 32,
                                description = 'Tam. Batch')

display(epochsWidget, batchWidget)

Execute abaixo para iniciar o treinamento.

In [ ]:
EPOCHS = epochsWidget.value
BATCH_SIZE = batchWidget.value

ytreino_onehot = tf.one_hot(ytreino, 12)
yteste_onehot = tf.one_hot(yteste, 12)

modelo.fit(xtreino, ytreino_onehot, epochs = EPOCHS, batch_size = BATCH_SIZE,
           validation_data = (xteste, yteste_onehot))

Após treinar a célula abaixo nos informará da acurácia do nosso modelo.

In [ ]:
score = modelo.evaluate(xteste, yteste_onehot, return_dict = True)['categorical_accuracy']
print(f'O modelo treinado teve uma acurácia de {100*(score):.2f}%')

As duas próximas células serão utilizadas para comparar as litologias reais nos poços com as previsões realizadas pelo modelo acima. Selecione um número entre 1 e 96 para ver o respectivo resultado para aquele poço.



In [ ]:
numeroDoPocoPred = widgets.IntSlider(min = 1, max = 96, step = 1,
                                 description = 'Número do poço')
print('Selecione abaixo o número do poço a ser previsto:')
display(numeroDoPocoPred)

In [ ]:
plotarPredicao(numeroDoPocoPred.value - 1, modelo, redeNeural = True)

Devemos lembrar que na estratégia de separação, consideramos todo o conjunto de dados. Logo, alguns poços visualizados acima todos podem conter todo o seu trecho utilizado durante o treino. Isso implica não podermos usá-los como comparativo para avaliar visualmente a performance do nosso modelo. 

Assim, execute as duas células abaixo para observar somente os dados de teste e suas respectivas previsões, comparando litologia a litologia. Note ainda que essa visualização não reflete de forma alguma estratigrafia ou a disposição das litologias em um poço específico.

In [ ]:
numeroLito = widgets.IntSlider(min = 1, max = 12, step = 1,
                                 description = 'Litologia:')
print('Escolha um número entre 1 e 12 refere à litologia, para comparação:')
display(numeroLito)

In [ ]:
plotarTeste(modelo, numeroLito.value - 1, redeNeural = True)

Vimos acima uma métrica numérica para avaliar a performance do nosso modelo: a acurácia. A acurácia compara o total de acertos com o número de exemplos na nosso conjunto de teste que está sendo avaliado através da fórmula:

<center> $ \frac {Total \: de \: acertos} {Total  \:  de  \:  exemplos}$ </center>

O problema dessa métrica é que ela avalia todas as classes/litologias ao mesmo tempo, mas não nos informa individualmente a eficiência de previsão de uma certa litologia. Por isso, é importante também utilizar as métricas Precisão, Recall e F1-Score, definidas do seguinte modo:

<center>
Precisão:

$ \frac {Positivos \: verdadeiros} {Positivos \: verdadeiros \: + \:Falsos \: verdadeiros}$

Recall:

$ \frac {Positivos \: verdadeiros} {Positivos \: verdadeiros \: + \:Falsos \: negativos}$

F1-Score:

$ 2 \cdot \frac {Precisao \:\cdot\: Recall} {Precisao \:+\: Recall}$

</center>

Quanto mais próximos de 1, melhor a eficiência de predição da litologia, e quanto mais próximo de 0, pior. Na dúvida, basta olhar para o F1-Score visto que o mesmo leva em consideração tanto precisão quanto recall para unificar as métricas.

In [ ]:
print(classification_report(yteste, np.argmax(modelo.predict(xteste), axis = 1),
                            target_names = lithology_keys.values()))

Uma outra forma de observar os resultados individualmente é a matriz de confusão. Execute a célula abaixo e vamos discutir o significado da matriz de confusão e como realizar sua leitura.

In [ ]:
confusionMatrix = confusion_matrix(yteste, np.argmax(modelo.predict(xteste), axis = 1))

fig, ax = plt.subplots(1, 1, figsize = (20 , 12))

sns.heatmap(confusionMatrix, cmap = 'GnBu', annot = True, square = True,
            ax = ax, fmt = 'g')
ax.set_xticklabels(list(lithology_keys.values()), rotation = 'vertical')
ax.set_yticklabels(list(lithology_keys.values()), rotation = 'horizontal')
ax.set_title('Matriz de confusão', fontsize = 14)
ax.set_xlabel('Previsto')
ax.set_ylabel('Verdadeiro');

## E agora, o que fazer com o modelo pronto?

Dependendo dos resultados obtidos, o que fazer com o modelo pronto vai depender em que segmento estamos: academia ou indústria. Caso os resultados não foram positivos, a modelagem ou mesmo os dados devem ser revistos. Vamos tentar disuctir alguns possíveis pontos.

In [ ]:
modelo.save('litho_pred.h5')